In [1]:
print("ok")

ok


In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [9]:
os.environ["OPENAI_API_KEY"]  = OPENAI_API_KEY

In [10]:
from langchain.document_loaders import PyPDFLoader

In [12]:
%cd ..

c:\Users\HP\Desktop\ML\7 Gen AI\Interview-Question-Creater


c:\Users\HP\Desktop\ML\7 Gen AI\Interview-Question-Creater\7_genai\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
%pwd

'c:\\Users\\HP\\Desktop\\ML\\7 Gen AI\\Interview-Question-Creater'

In [14]:
file_path = "./Data/stats.pdf"
loader = PyPDFLoader(file_path)

data = loader.load()


In [15]:
data

[Document(page_content='Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom\n-1\nto\n1,\nwith\na\nvalue\nof\n0\nindicating\nno\nlinear\nrelationship\nbetween\nthe\ntwo\nvariables,\na\nvalue\nof\n1\nindicating\na\nperfect\npositive\nlinear\nrelationship,\nand\na\nvalue\nof\n-1\nindicating\na\nperfect\nnegative\nlinear\nrelationship.\nCorrelation\nis\nimportant\nin\nstatistics\nbecause\nit\ncan\nbe\nused\nto\n1.\nIdentify\nrelationships\nbetween\nvariables:\nCorrelation\ncan\nbe\nused\nto\nidentify\nwhether\nthere\nis\na\nrelationship\nbetween\ntwo\nvariables,\nand\nif\nso,\nwhether\nthe\nrelationship\nis\npositive\nor\nnegative.\nThis\ninformation\ncan\nbe\nuseful\nfor\nunderstanding

In [16]:
len(data)

3

In [18]:
question_gen = ""
for page in data:
    question_gen+=page.page_content

In [19]:
question_gen

'Importance\nand\nthe\nuse\nof\ncorrelation\nin\nStatistics\nIntroduction\nCorrelation\nis\na\nstatistical\nmeasure\nthat\nexpresses\nthe\nextent\nto\nwhich\ntwo\nvariables\nare\nlinearly\nrelated.\nIt\nis\na\ncommon\ntool\nfor\ndescribing\nsimple\nrelationships\nwithout\nmaking\na\nstatement\nabout\ncause\nand\neffect.\nCorrelation\ncoefficients\nrange\nfrom\n-1\nto\n1,\nwith\na\nvalue\nof\n0\nindicating\nno\nlinear\nrelationship\nbetween\nthe\ntwo\nvariables,\na\nvalue\nof\n1\nindicating\na\nperfect\npositive\nlinear\nrelationship,\nand\na\nvalue\nof\n-1\nindicating\na\nperfect\nnegative\nlinear\nrelationship.\nCorrelation\nis\nimportant\nin\nstatistics\nbecause\nit\ncan\nbe\nused\nto\n1.\nIdentify\nrelationships\nbetween\nvariables:\nCorrelation\ncan\nbe\nused\nto\nidentify\nwhether\nthere\nis\na\nrelationship\nbetween\ntwo\nvariables,\nand\nif\nso,\nwhether\nthe\nrelationship\nis\npositive\nor\nnegative.\nThis\ninformation\ncan\nbe\nuseful\nfor\nunderstanding\nthe\nrelationships\nb

In [20]:
from langchain.text_splitter import TokenTextSplitter  #for chunk splitter
  

In [21]:
splitter_ques_gen = TokenTextSplitter(
    model_name= "gpt-3.5-turbo",
    chunk_size= 10000,
    chunk_overlap = 200
)
#Chunk creation

In [22]:
chunk_ques_gen = splitter_ques_gen.split_text(question_gen)


In [24]:
len(chunk_ques_gen)

1

In [25]:
from langchain.docstore.document import Document


In [26]:
document_ques_gen = [Document(page_content = t) for t in chunk_ques_gen]

In [27]:
splitter_ans_gen = TokenTextSplitter(
    model_name = 'gpt-3.5-turbo',
    chunk_size = 1000,
    chunk_overlap = 100
)

In [28]:
document_answer_gen = splitter_ans_gen.split_documents(
    document_ques_gen
)

In [29]:
from langchain.chat_models import ChatOpenAI

In [30]:
llm_ques_gen_pipeline = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature = 0.3
)

In [31]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [32]:
from langchain.prompts import PromptTemplate


In [33]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])


In [34]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [35]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [36]:
from langchain.chains.summarize import load_summarize_chain


In [37]:
ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline, 
                                          chain_type = "refine", 
                                          verbose = True, 
                                          question_prompt=PROMPT_QUESTIONS, 
                                          refine_prompt=REFINE_PROMPT_QUESTIONS)

In [38]:
ques = ques_gen_chain.run(document_ques_gen)

print(ques)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
Importance
and
the
use
of
correlation
in
Statistics
Introduction
Correlation
is
a
statistical
measure
that
expresses
the
extent
to
which
two
variables
are
linearly
related.
It
is
a
common
tool
for
describing
simple
relationships
without
making
a
statement
about
cause
and
effect.
Correlation
coefficients
range
from
-1
to
1,
with
a
value
of
0
indicating
no
linear
relationship
between
the
two
variables,
a
value
of
1
indicating
a
perfect
positive
linear
relationship,
and
a
value
of
-1
indicating
a
perfect
negative
linear
relationship.
Correlation
is
important
in
statistics
because
it
can
be
used
to
1.
Identify
relationships
between
variables:
Correlation
c

In [39]:
from langchain.embeddings.openai import OpenAIEmbeddings


In [40]:
embeddings = OpenAIEmbeddings()


In [41]:
from langchain.vectorstores import FAISS


In [42]:
vector_store = FAISS.from_documents(document_answer_gen, embeddings)


In [43]:
llm_answer_gen = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")


In [44]:
ques_list = ques.split("\n")


In [45]:
ques_list

['1. What is correlation in statistics and how is it measured?',
 '2. What is the range of correlation coefficients and what do values of 0, 1, and -1 indicate?',
 '3. How can correlation be used to identify relationships between variables?',
 '4. In what ways can correlation be used to make predictions in various fields?',
 '5. How is correlation used as a starting point for developing causal models?',
 '6. Give examples of how correlation is used in the fields of business, finance, medicine, and psychology.',
 '7. How can correlation be used in the real world by a marketing manager, financial analyst, medical researcher, and psychologist?',
 '8. Why is it important to note that correlation does not equal causation?',
 '9. Explain the example of the correlation between ice cream sales and shark attacks and why it does not imply causation.',
 '10. Overall, how is correlation described as a powerful statistical tool and what are its main uses in various fields?']

In [46]:
from langchain.chains import RetrievalQA

In [47]:
answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen, 
                                               chain_type="stuff", 
                                               retriever=vector_store.as_retriever())

In [48]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_generation_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")

Question:  1. What is correlation in statistics and how is it measured?
Answer:  Correlation in statistics is a measure that expresses the extent to which two variables are linearly related. It is a statistical tool used to describe simple relationships between variables without implying causation. Correlation coefficients range from -1 to 1, with 0 indicating no linear relationship, 1 indicating a perfect positive linear relationship, and -1 indicating a perfect negative linear relationship.
--------------------------------------------------\n\n
Question:  2. What is the range of correlation coefficients and what do values of 0, 1, and -1 indicate?
Answer:  The range of correlation coefficients is from -1 to 1. A correlation coefficient of 0 indicates no linear relationship between the two variables. A correlation coefficient of 1 indicates a perfect positive linear relationship, meaning as one variable increases, the other variable also increases in a linear fashion. A correlation co